# ***Блок подготовки библиотек***

### Импорт библиотек

In [8]:
import time
import math
import folium
import numpy as np
import pandas as pd
import networkx as nx
import matplotlib.pyplot as plt
from tqdm import tqdm
from scipy.spatial.distance import cdist
from sklearn_extra.cluster import KMedoids
from sklearn.preprocessing import StandardScaler
from sklearn.cluster import KMeans, MiniBatchKMeans
from ortools.constraint_solver import pywrapcp, routing_enums_pb2
from ortools.graph.python import min_cost_flow, max_flow, linear_sum_assignment

# ***Функции***

### Расчет расстояния между точками

In [10]:
def custom_distance(coord1, coord2):
    return haversine(coord1[0],coord1[1],coord2[0],coord2[1])

def haversine(lon1, lat1, lon2, lat2):
    """
    Calculate the great circle distance in kilometers between two points
    on the earth (specified in decimal degrees)
    """
    # convert decimal degrees to radians
    lon1, lat1, lon2, lat2 = map(math.radians, [lon1, lat1, lon2, lat2])
    # haversine formula
    dlon = lon2 - lon1
    dlat = lat2 - lat1
    a = math.sin(dlat/2)**2 + math.cos(lat1) * math.cos(lat2) * math.sin(dlon/2)**2
    c = 2 * math.asin(math.sqrt(a))
    r = 6371 # Radius of earth in kilometers
    return c * r * 1000

### Поиск оптимального количества кластеров и работа с кластерами



In [11]:
# Поиск оптимального количества кластеров, с которыми будем работать
def search_KMedoids(graph, n_start=3, n_end=50):
    nodes_coord = np.array([(coord['lon'], coord['lat']) for i, coord in graph.nodes(data=True)])
    scaler = StandardScaler().fit(nodes_coord)
    x_scaled = scaler.transform(nodes_coord)

    inertias = []
    for i in range(n_start, n_end+1):
        kMedoids = KMedoids(n_clusters=i, random_state=42, method='pam')
        kMedoids.fit(x_scaled)
        inertias.append(kMedoids.inertia_)
    plt.figure(figsize=(5, 3))
    plt.scatter([i for i in range(n_start, n_end+1)], inertias)


# Добавление номера кластера к узлу
def add_cluster_info_to_graph(graph, clusters):
    for cluster_num, nodes in clusters.items():
        for node in nodes:
            graph.nodes[node]['cluster_number'] = cluster_num
    return graph


# Разделение точек на кластеры
def kmedoids_clustering_for_graph(graph, num_clusters=5):
    nodes_coord = np.array([(coord['lon'], coord['lat']) for i, coord in graph.nodes(data=True)])
    scaler = StandardScaler().fit(nodes_coord)
    x_scaled = scaler.transform(nodes_coord)
    kmedoids = KMedoids(n_clusters=num_clusters, method='pam', random_state=42).fit(x_scaled)
    labels = kmedoids.labels_
    medoid_indices = kmedoids.medoid_indices_

    clusters = {}
    for i, label in enumerate(labels):
        if label not in clusters:
            clusters[label] = []
        clusters[label].append(i)

    output_graph = add_cluster_info_to_graph(graph, clusters)
    return output_graph, medoid_indices


# Выделение подграфа на основе кластера
def extract_cluster_subgraph(graph, cluster_number_to_extract):
    nodes_to_keep = [node for node, data in graph.nodes(data=True) if data['cluster_number'] == cluster_number_to_extract]
    subgraph = graph.subgraph(nodes_to_keep)
    return subgraph

### Вывод результата OR-Tools

In [12]:
def print_solution(data, manager, routing, solution):
    """Prints solution on console."""
    print(f"Objective: {solution.ObjectiveValue()}")
    max_route_distance = 0
    for vehicle_id in range(data["num_vehicles"]):
        index = routing.Start(vehicle_id)
        plan_output = f"Route for vehicle {vehicle_id}:\n"
        route_distance = 0
        while not routing.IsEnd(index):
            plan_output += f" {manager.IndexToNode(index)} -> "
            previous_index = index
            index = solution.Value(routing.NextVar(index))
            route_distance += routing.GetArcCostForVehicle(
                previous_index, index, vehicle_id
            )
        plan_output += f"{manager.IndexToNode(index)}\n"
        plan_output += f"Distance of the route: {route_distance}m\n"
        print(plan_output)
        max_route_distance = max(route_distance, max_route_distance)
    print(f"Maximum of the route distances: {max_route_distance}m - {max_route_distance/1000}km")



#
def get_routes(solution, routing, manager):
    """Get vehicle routes from a solution and store them in an array."""
    # Get vehicle routes and store them in a two dimensional array whose
    # i,j entry is the jth location visited by vehicle i along its route.
    routes = []
    for route_nbr in range(routing.vehicles()):
        index = routing.Start(route_nbr)
        route = [manager.IndexToNode(index)]
        while not routing.IsEnd(index):
            index = solution.Value(routing.NextVar(index))
            route.append(manager.IndexToNode(index))
        routes.append(route)
    return routes

### Поиск наименьшего расстояния между соседними кластерами

In [13]:
def search_min_weight_path(graph, clusters):
    transition = {}
    # Здесь идет перебор соседних кластеров, выделение подграфа,
    # удаление ребер между точками в одном кластере,
    # поиск наименьшего пути между точками
    for i in range(-1, len(correct_clusters) - 1):
        selected_nodes = [node for node, attr in work_graph.nodes(data=True) if attr['cluster_number'] in (correct_clusters[i], correct_clusters[i+1])]
        subgraph = work_graph.subgraph(selected_nodes).copy()
        for u, v in subgraph.edges():
            if subgraph.nodes[u]['cluster_number'] == subgraph.nodes[v]['cluster_number']:
                subgraph.remove_edge(u, v)
        node = min(subgraph.edges(data=True), key=lambda x: x[2]['weight'])
        if subgraph.nodes[node[0]]['cluster_number'] == correct_clusters[i]:
            transition[correct_clusters[i], correct_clusters[i+1]] = (node[0], node[1])
        else:
            transition[correct_clusters[i], correct_clusters[i+1]] = (node[1], node[0])

    # Данный словарь хранит начальную и конечную точку каждого кластера
    extreme_points = {}
    for i in range(-2, len(correct_clusters) - 2):
        extreme_points[correct_clusters[i+1]] = ([transition[(correct_clusters[i], correct_clusters[i+1])][1], transition[(correct_clusters[i+1], correct_clusters[i+2])][0]])

    return transition, extreme_points

### Поиск оптимального пути графа





In [14]:
# создание матрицы на основе полного графа
def create_matrix_from_graph(graph):
    # max_distance = 4294967295
    max_distance = 0
    matrix = np.full((len(graph.nodes()), len(graph.nodes())), max_distance)
    max_weight = max(graph.edges(data=True), key=lambda x: x[2]['weight'])
    for u, v, weight in graph.edges(data=True):
        weight = weight['weight']
        matrix[u][v] = weight
        matrix[v][u] = weight
    return matrix, max_weight


# создание словаря данных для OR-Tools
def create_data_model(graph, start=0, end=0):
    """Stores the data for the problem."""
    data = {}
    matrix_from_graph, max_weight = create_matrix_from_graph(graph)
    data["distance_matrix"] = matrix_from_graph
    data["num_vehicles"] = 1
    data["starts"] = [max_weight[0]]
    data["ends"] = [max_weight[1]]
    return data


def create_data_model_for_clusters(graph, start=0, end=0, extreme=False):
    """Stores the data for the problem."""
    data = {}
    matrix_G = matrix_from_graph(graph)
    nodes = [node for node, attr in graph.nodes(data=True)]
    index_mapping = {index: i for i, index in enumerate(nodes)}
    data["distance_matrix"] = matrix_G
    data["num_vehicles"] = 1
    if isinstance(extreme, list):
        data["starts"] = [index_mapping[extreme[0]]]
        data["ends"] = [index_mapping[extreme[1]]]
    else:
        max_index = np.unravel_index(np.argmax(matrix_G), matrix_G.shape)
        row_index, col_index = max_index
        data["starts"] = [int(row_index)]
        data["ends"] = [int(col_index)]
    return data


# Поиск оптимального пути на основе полного графа
def search_optimal_path(graph, clusters=False, timeout=False, dimension=False, extreme=False):
    if clusters:
        data = create_data_model_for_clusters(graph, extreme=extreme)
    else:
        data = create_data_model(graph)
    manager = pywrapcp.RoutingIndexManager(len(data["distance_matrix"]), data["num_vehicles"], data["starts"], data["ends"])
    routing = pywrapcp.RoutingModel(manager)

    def distance_callback(from_index, to_index):
        """Returns the distance between the two nodes."""
        # Convert from routing variable Index to distance matrix NodeIndex.
        from_node = manager.IndexToNode(from_index)
        to_node = manager.IndexToNode(to_index)
        return data["distance_matrix"][from_node][to_node]

    transit_callback_index = routing.RegisterTransitCallback(distance_callback)
    routing.SetArcCostEvaluatorOfAllVehicles(transit_callback_index)
    if dimension:
        dimension_name = "Distance"
        routing.AddDimension(
            transit_callback_index,
            0,  # no slack
            1800000,  # vehicle maximum travel distance
            True,  # start cumul to zero
            dimension_name,)
        distance_dimension = routing.GetDimensionOrDie(dimension_name)
        distance_dimension.SetGlobalSpanCostCoefficient(100)
    # Setting first solution heuristic.
    search_parameters = pywrapcp.DefaultRoutingSearchParameters()
    # search_parameters.first_solution_strategy = (routing_enums_pb2.FirstSolutionStrategy.PATH_CHEAPEST_ARC)
    if timeout:
        search_parameters.time_limit.seconds = timeout
    # solution = routing.SolveWithParameters(search_parameters)
    # if solution:
    # #     # print_solution(data, manager, routing, solution)
    #     routes = get_routes(solution, routing, manager)[0]
    search_parameters.local_search_metaheuristic = (routing_enums_pb2.LocalSearchMetaheuristic.SIMULATED_ANNEALING)
    solution = routing.SolveWithParameters(search_parameters)
    if solution:
        # print_solution(data, manager, routing, solution)
        routes = get_routes(solution, routing, manager)[0]
    return routes, solution.ObjectiveValue()

### Создание графа из df

In [15]:
# Данный граф создает все маршруты между точками.
def create_graph_from_coordinates(df, threshold_distance=1700000, distance_bool=False):
    '''Создание графа между точками
       threshold_distance - максимальное расстояние между точками для создания узла (в метрах)
       distance           - учитывать ли дистанцию между точками
    '''
    G = nx.Graph()
    coordinates = list(zip(df['lon'], df['lat']))
    total_coordinates = len(coordinates)

    for idx, coord in enumerate(coordinates):
        G.add_node(idx, lon=coordinates[idx][0], lat=coordinates[idx][1])

    for i in tqdm(range(total_coordinates), desc="Progress"):
        for j in range(i + 1, total_coordinates):
            distance = int(haversine(coordinates[i][0], coordinates[i][1], coordinates[j][0], coordinates[j][1]))
            if distance_bool:
                if distance <= threshold_distance:
                    G.add_edge(i, j, weight=distance)
            else:
                G.add_edge(i, j, weight=distance)
    return G

### Сборка графа на основе точек по поиска пути

In [16]:
def create_graph_after_solver(graph, routes):
    new_G = nx.Graph()
    for node in routes:
        # Копирование атрибутов узла из оригинального графа G в новый граф new_G
        if node in graph.nodes():
            new_G.add_node(node, **graph.nodes[node])
    for i in range(len(routes) - 1):
        node1 = routes[i]
        node2 = routes[i + 1]
        if graph.has_edge(node1, node2):
            # Копирование атрибутов ребра из оригинального графа G в новый граф new_G
            new_G.add_edge(node1, node2, **graph.get_edge_data(node1, node2))
        # else:
        #     new_G.add_edge(node1, node2, weight=40000000)
    return new_G

### Матрица из графа

In [17]:
def matrix_from_graph(graph):
    return nx.floyd_warshall_numpy(graph).astype(np.int32)

### Создание матрицы по кластерам

In [18]:
def create_graph_after_solver_full(graph, full_graph, routes):
    for node in routes:
        # Копирование атрибутов узла из оригинального графа G в новый граф new_G
        if node in graph.nodes():
            full_graph.add_node(node, **graph.nodes[node])
    for i in range(len(routes) - 1):
        node1 = routes[i]
        node2 = routes[i + 1]
        if graph.has_edge(node1, node2):
            # Копирование атрибутов ребра из оригинального графа G в новый граф new_G
            full_graph.add_edge(node1, node2, **graph.get_edge_data(node1, node2))
        # else:
        #     new_G.add_edge(node1, node2, weight=40000000)
    return full_graph

### Создание интерактивной карты на основе графа

In [19]:
def create_map_from_graph(G):
    '''Создание интерактивной карты из графа. Граф должен иметь lat= и lon='''

    m = folium.Map(location=[56.322, 90.004], zoom_start=3)

    node_positions = {}
    node_positions = {node: (G.nodes[node]['lat'], G.nodes[node]['lon']) for node in G.nodes()}

    for node, pos in node_positions.items():
        folium.CircleMarker(location=pos, radius=0.1, color='blue', fill=True).add_to(m)


    for u, v, weight in G.edges(data=True):
        weight = weight['weight']
        pos_u = node_positions[u]
        pos_v = node_positions[v]
        if not weight:
            folium.PolyLine(locations=[pos_u, pos_v], color='red', weight=0.9, opacity=0.5).add_to(m)
        else:
            folium.PolyLine(locations=[pos_u, pos_v], color='black', weight=0.9, opacity=0.5).add_to(m)

    return m

# ***Подготовка данных***

### Чтение df

In [20]:
df = pd.read_csv('random_points.csv')
df = df.dropna()[['Долгота', 'Широта']].reset_index(drop=True)
df.columns = ['lon', 'lat']

In [21]:
work_df = df[::1000].copy()                # Если брать каждую (127*целое число) точку, то я захватываю всю территорию России
work_df = work_df.reset_index(drop=True)

### Создание графа

In [22]:
# Данный граф создает все маршруты между точками.
def create_graph_from_coordinates(df, threshold_distance=1700000, distance_bool=False):
    '''Создание графа между точками
       threshold_distance - максимальное расстояние между точками для создания узла (в метрах)
       distance           - учитывать ли дистанцию между точками
    '''
    G = nx.Graph()
    coordinates = list(zip(df['lon'], df['lat']))
    total_coordinates = len(coordinates)

    for idx, coord in enumerate(coordinates):
        G.add_node(idx, lon=coordinates[idx][0], lat=coordinates[idx][1])

    for i in tqdm(range(total_coordinates), desc="Progress"):
        for j in range(i + 1, total_coordinates):
            distance = int(haversine(coordinates[i][0], coordinates[i][1], coordinates[j][0], coordinates[j][1]))
            if distance_bool:
                if distance <= threshold_distance:
                    G.add_edge(i, j, weight=distance)
            else:
                G.add_edge(i, j, weight=distance)
    return G

In [23]:
G = create_graph_from_coordinates(work_df)

Progress: 100%|████████████████████████████████████████████████████████████████████| 122/122 [00:00<00:00, 2579.35it/s]


# ***Тестовая область***

In [24]:
df = pd.read_csv('random_points.csv')
df = df.dropna()[['Долгота', 'Широта']].reset_index(drop=True)
df.columns = ['lon', 'lat']

In [25]:
work_df = df[::250].copy()                # Если брать каждую (127*целое число) точку, то я захватываю всю территорию России
work_df = work_df.reset_index(drop=True)
G = create_graph_from_coordinates(work_df)

Progress: 100%|█████████████████████████████████████████████████████████████████████| 485/485 [00:00<00:00, 708.88it/s]


### Глобальный граф

In [27]:
start_time = time.time()

timeout=5
routes, v = search_optimal_path(G, timeout=timeout)
solver_graph = create_graph_after_solver(G, routes)


end_time = time.time()
execution_time = end_time - start_time
print(f"Время выполнения: {execution_time:.2f} секунд")
print(f'{v//1000} km')

Время выполнения: 5.32 секунд
33606 km


### Локальный граф

In [28]:
dista = []
for i in range(5, 6):
    start_time = time.time()
    n_cl = i
    timeout = 2
    # timeout = timeout // (n_cl*8)
    # print(timeout)

    work_graph, medoids = kmedoids_clustering_for_graph(G, n_cl)
    med_graph = G.subgraph(medoids)
    cluster_routes, b = search_optimal_path(med_graph, clusters=True, timeout=1)

    d = {}
    correct_clusters = []
    correct_nodes = []
    for i, node in enumerate(med_graph.nodes(data=True), 0):
        d[i] = (node[0], node[1]['cluster_number']) # {index: (номер ноды, номер кластера)}
    for i in cluster_routes:
        # перебирая словарь, выделяем список кластеров по порядку и нод по порядку.
        # Кластеры нужны, чтобы по ним проходиться i и i+1
        # Ноды нужны для функции create_graph_after_solver, чтобы выделить последовательность пути
        correct_nodes.append(d[i][0])
        correct_clusters.append(d[i][1])

    transition, extreme_points = search_min_weight_path(work_graph, correct_clusters)
    full_graph = nx.Graph()
    full_path = 0
    for i in range(len(correct_clusters)):
        selected_nodes = [node for node, attr in work_graph.nodes(data=True) if attr['cluster_number'] == correct_clusters[i]]
        sub = work_graph.subgraph(selected_nodes).copy()
        cluster_routes_sub, path = search_optimal_path(sub, clusters=True, timeout=timeout, extreme=extreme_points[correct_clusters[i]])
        full_path += path
        d_sub = {}
        correct_clusters_sub = []
        correct_nodes_sub = []
        for i, node in enumerate(sub.nodes(data=True), 0):
            d_sub[i] = (node[0], node[1]['cluster_number']) # {index: (номер ноды, номер кластера)}

        for i in cluster_routes_sub:
            # перебирая словарь, выделяем список кластеров по порядку и нод по порядку.
            # Кластеры нужны, чтобы по ним проходиться i и i+1
            # Ноды нужны для функции create_graph_after_solver, чтобы выделить последовательность пути
            correct_nodes_sub.append(d_sub[i][0])
            correct_clusters_sub.append(d_sub[i][1])
        full_graph = create_graph_after_solver_full(sub, full_graph, correct_nodes_sub)
    for j, i in transition.items():
        if work_graph.has_edge(i[0], i[1]) and j != transition[(correct_clusters[-1], correct_clusters[0])]:
            full_path += work_graph.get_edge_data(i[0], i[1])['weight']
            # Копирование атрибутов ребра из оригинального графа G в новый граф new_G
            full_graph.add_edge(i[0], i[1], **work_graph.get_edge_data(i[0], i[1]))
    full_graph.remove_edge(*transition[(correct_clusters[-1], correct_clusters[0])])
    print(f'{full_path}m - {full_path//1000} km')
    # print(f"{full_path - work_graph.get_edge_data(*transition[(correct_clusters[-2], correct_clusters[-1])])['weight']}m - {(full_path - work_graph.get_edge_data(*transition[(correct_clusters[-2], correct_clusters[-1])])['weight'])//1000} km")
    print('Количество кластеров: ',n_cl)
    end_time = time.time()
    execution_time = end_time - start_time
    print(f"Время выполнения: {execution_time:.2f} секунд", '\n')

41484433m - 41484 km
Количество кластеров:  5
Время выполнения: 13.15 секунд 



In [30]:
create_map_from_graph(solver_graph)

In [31]:
create_map_from_graph(full_graph)

In [32]:
create_map_from_graph(med_graph)

In [34]:
def create_map_from_graph_with_clusters_color(G):
    '''Создание интерактивной карты из графа. Граф должен иметь lat= и lon='''

    m = folium.Map(location=[56.322, 90.004], zoom_start=3)

    node_positions = {}
    node_positions = {node: (G.nodes[node]['lat'], G.nodes[node]['lon'], G.nodes[node]['cluster_number']) for node in G.nodes()}
    cluster_colors = {
    0: 'purple',
    1: 'red',
    2: 'blue',
    3: 'green',
    4: 'orange',}


    for node, pos in node_positions.items():
        cluster_number = pos[2]
        popup_info = f"Node: {node}, Cluster: {cluster_number}"
        color = cluster_colors.get(cluster_number, 'black')  # Используйте черный цвет по умолчанию
        folium.CircleMarker(location=pos[:2], radius=3, color=color, fill=True, popup=popup_info).add_to(m)


    for u, v, weight in G.edges(data=True):
        weight = weight['weight']
        pos_u = node_positions[u]
        pos_v = node_positions[v]
        if not weight:
            folium.PolyLine(locations=[pos_u[:2], pos_v[:2]], color='red', weight=0.9, opacity=0.5).add_to(m)
        else:
            folium.PolyLine(locations=[pos_u[:2], pos_v[:2]], color='black', weight=1.5, opacity=0.5).add_to(m)

    return m

In [35]:
create_map_from_graph_with_clusters_color(full_graph)